# Inheritance

## Subclassing built in types
Not always we get what we expect because built in methods often do not call user defined methods because they are written in C.

In [1]:
# Our custom __setitem__ is ignored by the built in dict methods

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

dd = DoppelDict(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three = 3)
print(dd)


{'one': 1}
{'one': 1, 'two': [2, 2]}
{'one': 1, 'two': [2, 2], 'three': 3}


This is a violation of the basic rule of object-oriented programming

In [2]:
# Example the same problem happens when calling overriden methods of other classes using BIF methods

class AnswerDict(dict):
    def __getitem__(self,key):
        return 42

ad = AnswerDict(a='foo')
print(ad['a'])
d = {}
d.update(ad)
print(d['a'])
print(d)

42
foo
{'a': 'foo'}


Thus instead of directly subclassing the built in types, subclass `UserDict`, `UserList` and `UserString` from `collections`.

In [4]:
# Example subclassing from collections: everything works!

import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        return super().__setitem__(key, [value] * 2)
    
dd = DoppelDict2(one = 1)
dd['two'] = 2
dd.update(three = 3)
print(dd)

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [6]:
# Similar example to the above

import collections

class AnswerDict2(collections.UserDict):
    def __getitem__(self, _):
        return 42
    
ad = AnswerDict2(a = 3)
print(ad['a'])
d = {}
d.update(ad)
print(d)


42
{'a': 42}


## Multiple inheritance and method resolution order (mro)
How does Python handle the "diamond problem"?

In [8]:
# Diamond problem in Python

class A:
    def ping(self):
        print('ping', self)

class B(A):
    def pong(self):
        print('pong', self)

class C(A):
    def pong(self):
        print('PONG', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


You can specify explicitly which method you want to use by specifying which class to use.

In [9]:
d = D()
d.pong()
C.pong(d)

pong <__main__.D object at 0x0000028FD2762A50>
PONG <__main__.D object at 0x0000028FD2762A50>


If you do not specify this explicitly, then Python has its own rules and follows a specific order called MRO (method resolution order). This can be seen in the `__mro__` attribute. <br>
`super()` follows the MRO when invoking a method.

In [10]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [11]:
d = D()
d.ping()
d.pingpong()

ping <__main__.D object at 0x0000028FD2A24CD0>
post-ping: <__main__.D object at 0x0000028FD2A24CD0>
ping <__main__.D object at 0x0000028FD2A24CD0>
post-ping: <__main__.D object at 0x0000028FD2A24CD0>
ping <__main__.D object at 0x0000028FD2A24CD0>
pong <__main__.D object at 0x0000028FD2A24CD0>
pong <__main__.D object at 0x0000028FD2A24CD0>
PONG <__main__.D object at 0x0000028FD2A24CD0>


## Coping with multiple inheritance
Check the book